In [9]:
import os
import sys
#sys.path.append("/home/infomax/svc/lib")
import datetime
import requests
import logging
import time
import re
import lxml
from decimal import Decimal
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup

import pandas as pd

In [10]:
ops = webdriver.ChromeOptions()
#ops.add_argument('headless')
ops.add_argument('window-size=1920x860')
ops.add_argument('window-position=500,0')
#ops.add_argument("disable-gpu")
ops.add_argument(
"user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
ops.add_argument("lang=ko_KR")  # 한국어
ops.add_experimental_option("prefs", {
"profile.default_content_setting_values.automatic_downloads": True,
"directory_upgrade": True,
"safebrowsing.enabled": True})

In [11]:
with webdriver.Chrome(chrome_options=ops) as mydr:
    mydr.implicitly_wait(20)
#https://www.youtube.com/watch?v=j45mjlHNnKs
#https://www.youtube.com/watch?v=x6Y8S1msF14
#https://www.youtube.com/watch?v=Py5P4Ufv16Q
#https://www.youtube.com/watch?v=gwMa6gpoE9I #방탄 큰거
#mydr.get("https://www.youtube.com/watch?v=Py5P4Ufv16Q")

    mydr.get("https://www.youtube.com/watch?v=_cxvu6koHPw")
    try:
        # 로딩까지 wait
        element_present = EC.presence_of_element_located((By.ID,'more'))
        WebDriverWait(mydr, 20).until(element_present)

    except TimeoutException:
        pass
    #oTitle = mydr.find_element_by_xpath('//h1[@class = "title style-scope ytd-video-primary-info-renderer"]')
    #oTitle.click()
    last_page_height = mydr.execute_script("return document.documentElement.scrollHeight")/3
    mydr.execute_script(f"window.scrollTo(0, {last_page_height});")

    print("wait for comments to load ...")
    WebDriverWait(mydr, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "ytd-comments ytd-comment-simplebox-renderer"))
    )
    oBtn_More = mydr.find_element_by_xpath('//paper-button[@id="more"]')
    oBtn_More.click()

    '''
    try:
        # 로딩까지 wait
        element_present = EC.presence_of_element_located((By.ID, 'sort-menu'))
        WebDriverWait(mydr, 5).until(element_present)
    except TimeoutException:
        pass
    '''
    last_size = 0
    nCount = 100
    for i in range(0,nCount):
        #new_page_height = mydr.execute_script("return document.documentElement.scrollHeight")/2
        lst_element = mydr.find_elements_by_xpath('//ytd-comment-renderer[@id="comment"]')
        last_element = lst_element[-1]
        mydr.execute_script("arguments[0].scrollIntoView(true);", last_element);
        #time.sleep(2.0)
        new_size = len(lst_element)
        print(i,end=' ')
        if new_size == last_size:
            print(last_size)
            print(new_size)
            break
        last_size = new_size

    print('코멘트 클릭 시작')

    lst_more = mydr.find_elements_by_xpath('//paper-button[@id="more"]')
    for data in lst_more:
        if '자세히' in data.text:
            mydr.execute_script("arguments[0].scrollIntoView(true);", data);
            mydr.execute_script("arguments[0].click();", data);
            time.sleep(1)
            #data.send_keys(Keys.Endter)

    print('코멘트 클릭 끝\n')

    sScripts = mydr.page_source

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


wait for comments to load ...
0 1 20
20
코멘트 클릭 시작
코멘트 클릭 끝



In [14]:
parse = BeautifulSoup(sScripts,'lxml')

sYoutube_Title = parse.find('meta', {'property': 'og:title'})['content']
sYoutube_URL = parse.find('meta', {'property': 'og:url'})['content']
sYoutube_Image = parse.find('meta', {'property': 'og:image'})['content']

sYoutube_Count = parse.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text
rx = re.compile(r'([조회수 ,회])')
sYoutube_Count = rx.sub('',sYoutube_Count)

sYoutube_Date = parse.find('div',{'id':'date'}).find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text
rx = re.compile(r'([ ])')
sYoutube_Date = rx.sub('',sYoutube_Date)

nUnit_Like = 1
nUnit_Dislike = 1
sYoutube_Like = parse.find('yt-formatted-string',{'aria-label':re.compile(r'좋아요')}).text
rx = re.compile(r'([천 만])')
if '천' in sYoutube_Like:
    sYoutube_Like = rx.sub('',sYoutube_Like)
    nUnit_Like = 1000
elif '만' in sYoutube_Like:
    sYoutube_Like = rx.sub('',sYoutube_Like)
    nUnit_Like = 10000
nYoutube_Like = int(float(sYoutube_Like) * nUnit_Like)


sYoutube_Dislike = parse.find('yt-formatted-string', {'aria-label': re.compile(r'싫어요')}).text
if '천' in sYoutube_Dislike:
    sYoutube_Dislike = rx.sub('',sYoutube_Dislike)
    nUnit_Dislike = 1000
elif '만' in sYoutube_Dislike:
    sYoutube_Dislike = rx.sub('',sYoutube_Dislike)
    nUnit_Dislike = 10000
nYoutube_Dislike = int(float(sYoutube_Dislike)*nUnit_Dislike)


#sYoutube_Discription = parse.find('meta', {'property': 'og:description'})['content']


'''
print('제목: ' + sYoutube_Title)
print('URL: ' + sYoutube_URL)
print('Image_URL: ' + sYoutube_Image)
print('조회수: ' + sYoutube_Count)
print('게시일: ' + sYoutube_Date)
print('좋아요: ' + str(nYoutube_Like))
print('싫어요: ' + str(nYoutube_Dislike))
'''
pdVideo_Info = pd.DataFrame({'sYoutube_Title':[sYoutube_Title],
                             'sYoutube_URL':[sYoutube_URL],
                             'sYoutube_Image':[sYoutube_Image],
                             'sYoutube_Count':[sYoutube_Count],
                             'sYoutube_Date':[sYoutube_Date],
                             'nYoutube_Like':[sYoutube_Like],
                             'nYoutube_Dislike':[sYoutube_Dislike]
                             })
pdVideo_Info

,sYoutube_Title,sYoutube_URL,sYoutube_Image,sYoutube_Count,sYoutube_Date,nYoutube_Like,nYoutube_Dislike
0,베테랑 형사팀 케미 모음zip 2부 [뭅뭅픽] 베테랑 l 황정민 l 장윤주 l 오달수,https://www.youtube.com/watch?v=_cxvu6koHPw,https://i.ytimg.com/vi/_cxvu6koHPw/maxresdefau...,90176,2020.2.23.,1.2,47


In [15]:
sYoutube_Channel_Nm = parse.find('ytd-channel-name',{'id':'channel-name'}).find('a').text

sYoutube_Channel_Sub_Count = parse.find('yt-formatted-string',{'id':'owner-sub-count'}).text
rx = re.compile(r'([구독자 만명])')
nUnit = 1
if '천' in sYoutube_Channel_Sub_Count:
    sYoutube_Channel_Sub_Count = rx.sub('', sYoutube_Channel_Sub_Count)
    nUnit = 1000
elif '만' in sYoutube_Channel_Sub_Count:
    sYoutube_Channel_Sub_Count = rx.sub('', sYoutube_Channel_Sub_Count)
    nUnit = 10000
nYoutube_Channel_Sub_Count = int(float(sYoutube_Channel_Sub_Count) * nUnit)

sYoutube_Discription = parse.find('div', {'id': 'description'}).text
sYoutube_Category = parse.find(text=re.compile(r'^카테고리')).find_parent().find_parent().find_next_sibling().find('a', {'class': 'yt-simple-endpoint style-scope yt-formatted-string'}).text
lst_Youtube_tag = parse.find_all('meta',{'property':'og:video:tag'})
sYoutube_tag = ', '.join(data['content'] for data in lst_Youtube_tag)
'''
print('채널 명: ' + sYoutube_Channel_Nm)
print('구독자:' + str(nYoutube_Channel_Sub_Count))
print('Description: ' +sYoutube_Discription)
print('카테고리 :' + sYoutube_Category)     
print('tag: ' + sYoutube_tag)
'''
pdChannel_Info = pd.DataFrame({'sYoutube_Channel_Nm': [sYoutube_Channel_Nm],
                               'nYoutube_Channel_Sub_Count': [nYoutube_Channel_Sub_Count],
                               'sYoutube_Description': [sYoutube_Discription],
                               'sYoutube_Category': [sYoutube_Category],
                               'sYoutube_tag': [sYoutube_tag]
                               })
pdChannel_Info

,sYoutube_Channel_Nm,nYoutube_Channel_Sub_Count,sYoutube_Description,sYoutube_Category,sYoutube_tag
0,뭅뭅,293000,누가 우리 막내한테 칼침 놨어?!!!!\n베테랑 속 형사팀 케미 모음.zip 2부\...,영화/애니메이션,"베테랑, 베테랑명장면, 베테랑명대사, 베테랑다시보기, 영화베테랑, 영화추천, 영화리..."


In [16]:
pd_lstComments = pd.DataFrame({'sComments_Author': [],
                           'sComments_Time': [],
                           'sComments_text': [],
                           'sComments_Like': []
                           })
lst_comments = parse.find_all('ytd-comment-thread-renderer',{'class':'style-scope ytd-item-section-renderer'})
        
#print('======================================================')
for data in lst_comments:
    sAuthor = data.find('a',{'id':'author-text'}).text.strip()
    sTime = data.find('yt-formatted-string', {'class': re.compile('published-time-text')}).text.strip()
    sComment = data.find('yt-formatted-string',{'id':'content-text'}).text.replace('\n',' ').strip()
    sLike = data.find('span',{'id':'vote-count-middle'}).text.strip()
    rx = re.compile(r'([천 만])')
    nUnit_Like = 1
    if '천' in sLike:
        sLike = rx.sub('',sLike)
        nUnit_Like = 1000
    elif '만' in sLike:
        sLike = rx.sub('',sLike)
        nUnit_Like = 10000
    nLike = int(float(sLike) * nUnit_Like)
    #print(sAuthor)
    #print(sTime)
    #print(sComment)
    #print("좋아요: " + str(nLike))
    #print('======================================================')

    pdComments = pd.DataFrame({'sComments_Author': [sAuthor],
                           'sComments_Time': [sTime],
                           'sComments_text': [sComment],
                           'sComments_Like': [sLike]
                           })
    pd_lstComments = pd_lstComments.append(pdComments)
pd_lstComments.index = range(len(pd_lstComments))    
pd_lstComments
   

,sComments_Author,sComments_Time,sComments_text,sComments_Like
0,메테오부르는북극곰,4시간 전(수정됨),명장면 명대사 모음 0:20 야ㅏㅏㅏㅏㅏㅏ아 우리 막내한테 칼침 논 새끼가 누구야...,13
1,김민소,1시간 전(수정됨),"장윤주 발차기 ㄹㅈㄷ,, 모델이라서 그런지 다리길이가 볼때마다 놀라운데 그 길이로...",28
2,돌 돌,3시간 전,0:20 이야야ㅑ아아아앗!!!!,46
3,최민기,4시간 전,세리 비서가 경찰이었구나,31
4,김승민,2시간 전,난 레알 멋있는게 막내 칼 맞고 빡쳐서 때리는데 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ ㅅㅂ 대장이 와...,99
5,o r,3시간 전,5:45 형사 쭈굴미 ㅋㅋㅋㅋㅋㅋㅋㅋ,75
6,MAME MY,3시간 전,7:08 맨앞에 형사 총받고 쓰러지는겨???ㅋㅋㅋㅋㅋㅋ,95
7,치치,4시간 전,얘 빽은 하나님빽이냐 하는 거 좋았는데 빠져서 아쉽다,131
8,장윤석,3시간 전,경찰서 뚱띠경찰 귀여워죽겟닼ㅋㅋㅋ,37
9,정제하,3시간 전(수정됨),4:15 진짜 카리스마는 이런거지!! 다른거 다 참아도 내새끼 건드는건 못참는다!!...,43
